# 포스트 리스트 URL


![3.jpg](images/3.jpg)

### 블로그 리스트 전체보기를 새탭으로 띄우기

블로그의 글 전체보기는 모든 블로그에 있는 링크 입니다. 이 링크 위에 마우스를 오버 해보면 실제 전체보기의 주소를 브라우저 좌측 하단을 통해 알 수 있습니다. 이 주소를 새로운 탭으로 띄우기 위해 Ctrl + 클릭해서 브라우저의 새 탭으로 해당 페이지를 직접 접속하도록 합니다.

![5.jpg](images/5.jpg)


### 실제 블로그 리스트를 불러오는 페이지 추적

새로운 탭으로 PostList.nhn 페이지가 열리면 개발자 도구의 1.네트워크 탭에서 2.기존의 기록이 있으면 기록을 모두 지우고 3.목록 열기를 클릭하면 어떤 페이지를 통해 블로그 포스트의 전체 목록을 가져오는지 확인 할 수 있습니다.

![6.jpg](images/6.jpg)

### 블로그 리스트

위에서 분석한 결과로 실제 블로그 포스트의 전체보기는 PostTitleAsync.nhn 주소에서 가져오는것을 확인 할 수 있으며 해당주소에 파라메타로 게시물 출력갯수, 카테고리 등을 넘겨서 원하는 형태로 리스트를 구해올 수 있음을 확인 할 수 있습니다.

![7.jpg](images/7.jpg)

### 필요정보

위에서 얻은 데이터에서 우리가 필요한 데이터는 다음페이지가 존재하는지를 판단하기 위해 pagingHtml 데이터와 게시물 포스트의 인덱스(longNo) 목록인 PostList 데이터가 필요합니다. 이를 구하는 코드를 작성해봅니다.

In [ ]:
import requests
import hjson
from bs4 import BeautifulSoup
import urllib

naver_id = "nkj2001"
page = 1

while True:
    url = "https://blog.naver.com/PostTitleListAsync.nhn?blogId={}&viewdate=&currentPage={}&categoryNo=&parentCategoryNo=&countPerPage=30".format(naver_id, page)
    r = requests.get(url)
    _json = hjson.loads(r.text)
    _posts = _json.get("postList")
    _paging = _json.get("pagingHtml")

    for _p in _posts:
        _logNo = _p.get("logNo")
        _title = urllib.request.unquote(_p.get("title")).replace("+", " ")
        _category_no = _p.get("categoryNo")
        _parent_category_no = _p.get("parentCategoryNo")
        _add_date = _p.get("addDate")
        _category = category[_category_no]

        print(_logNo, _title, _category)

    bs = BeautifulSoup(_paging, "lxml")
    _next_tag = bs.select_one("a.next")
    if _next_tag is None:
        break
    page += 1


### 필요정보 파싱
postList 에 있는 데이터를 받아서 루프를 돌며 logno, title, category_no 등의 필요한 데이터를 추출합니다. _parent_category_no 등은 사용하지는 않지만 차후 기능을 추가하거나 하는 경우 필요할지 모르니 일단 구해놓습니다. 여기서 구한 category_no 값을 통해 이전에 작성해놓은 dict 형태의 category 값에 대입하여 실제 카테고리명을 구합니다. 또한 urlencode 된 한글 제목을 실제로 알아보기 위해 urldecode를 해주기 위해 urllib 라이브러리에서 unquote() 함수를 사용합니다.

### 페이징 처리

_paging 이란 변수에 pagingHtml 값을 저장 후 이를 BeautifulSoup 으로 처리하여 a 태그의 next 클래스명의 존재 여부를 확인합니다. 해당 요소가 있으면 다음 페이지가 있는거고 없으면 마지막 페이지로 판단하여 루프를 탈출 합니다.

### 포스트 주소

![8.jpg](images/8.jpg)

실제 게시물 주소를 알기 위해 아무 포스트 하나를 선택하여 아까처럼 Ctrl + 클릭으로 새탭으로 링크를 엽니다.

### 상세 페이지 분석

![9.jpg](images/9.jpg)

새로운 페이지가 열리면 실제 블로그 포스트는 PostView.nhn 이라는 주소를 통해 보이는것을 확인 할 수 있으며 개발자 도구를 통해 블로그 포스트의 본문 내용이 div 태그에 se-main-container 클래스를 사용하는 것을 확인 할 수 있습니다. 그런데 여기서 주의해야할 한 가지가 현재 네이버 블로그는 2가지의 에디터를 제공하는데 최신버전인 스마트에디터 ONE 을 사용한 경우에는 위와 같은 클래스명을 쓰고 이전 버전인 스마트에디터 2.0 버전을 써서 글을 작성한 경우에는 div 태그의 postViewArea 라는 클래스를 사용합니다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import hjson
import urllib


naver_id = "nkj2001"

url = "https://blog.naver.com/WidgetListAsync.nhn?blogId={}&enableWidgetKeys=challengeMasterTrack_3%2CchallengeMasterTrack_2%2C1282902%2Csearch%2Ctitle%2Cmenu%2Cprofile%2Ccategory%2C1296453%2C1296454%2Cbuddyconnect%2Ccounter%2Cvisitorgp%2Crss%2C431894%2Cstat%2Ccontent%2Cgnb%2Cexternalwidget%2Cmusic".format(naver_id)
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://blog.naver.com/PostList.nhn?blogId={}&categoryNo=0&from=postList".format(naver_id)
}

category = {}
_r = requests.get(url, headers=header)
_json = hjson.loads(_r.text)
_category_str = _json.get("category").get("content")
bs = BeautifulSoup(_category_str, "lxml")
links = bs.select("a")
for l in links:
    _href = l.get("href")
    if _href is None or _href == "#":
        continue
    _text = l.text
    _id = l.get("id")
    _id = _id.replace("category", "")
    if category.get(_id) is None:
        category[_id] = _text

page = 1
while True:
    url = "https://blog.naver.com/PostTitleListAsync.nhn?blogId={}&viewdate=&currentPage={}&categoryNo=&parentCategoryNo=&countPerPage=30".format(naver_id, page)
    r = requests.get(url)
    _json = hjson.loads(r.text)
    _posts = _json.get("postList")
    _paging = _json.get("pagingHtml")

    for _p in _posts:
        _logNo = _p.get("logNo")
        _title = urllib.request.unquote(_p.get("title")).replace("+", " ")
        _category_no = _p.get("categoryNo")
        _parent_category_no = _p.get("parentCategoryNo")
        _add_date = _p.get("addDate")
        _category = category[_category_no]

        url = "https://blog.naver.com/PostView.nhn?blogId={}&logNo={}".format(naver_id, _logNo)

        r = requests.get(url)
        bs = BeautifulSoup(r.text, "lxml")
        contents = bs.select_one("div#postViewArea > div")
        if contents is None:
            contents = bs.select_one("div.se-main-container")
        if contents is not None:
            print(_title, _category, contents[0:10])
    
    bs = BeautifulSoup(_paging, "lxml")
    _next_tag = bs.select_one("a.next")
    if _next_tag is None:
        break
    page += 1
